# Import Modules

In [1]:
# Import modules
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

# Defining global variables

In [2]:
global_url_modules = "https://registry.terraform.io/v2/modules"
global_url_provider = "https://registry.terraform.io/v2/providers"

# Defining functions

In [3]:
# Returns from the given URL the pagination data from JSON
def get_pagination(url):
    response = requests.get(url)
    data = json.loads(response.text)
    return data['meta']['pagination']

In [4]:
# Returns
def get_modules(url, pages):
    local_list_modules = []
    for i in range(pages):
        url_page = url + "?page%5Bnumber%5D="+ str(i+1) + "&page%5Bsize%5D=15"
        response = requests.get(url_page)
        data = json.loads(response.text)
        print(i)
        # convert json data to list
        for item in data['data']:
            local_list_modules.append(item)

    return local_list_modules

In [5]:
def get_first_page(url):
    url = url + "?page%5Bnumber%5D="+ str(2) + "&page%5Bsize%5D=15"
    response = requests.get(url)
    data = json.loads(response.text)
    return data

get_first_page(global_url_modules)

{'data': [{'type': 'modules',
   'id': '1139',
   'attributes': {'downloads': 3547093,
    'full-name': 'cloudposse/ecs-container-definition/aws',
    'name': 'ecs-container-definition',
    'namespace': 'cloudposse',
    'owner-name': '',
    'provider-logo-url': '/images/providers/aws.png',
    'provider-name': 'aws',
    'source': 'https://github.com/cloudposse/terraform-aws-ecs-container-definition',
    'verified': False},
   'links': {'self': '/v2/modules/1139'}},
  {'type': 'modules',
   'id': '61',
   'attributes': {'downloads': 3347316,
    'full-name': 'terraform-aws-modules/ec2-instance/aws',
    'name': 'ec2-instance',
    'namespace': 'terraform-aws-modules',
    'owner-name': '',
    'provider-logo-url': '/images/providers/aws.png',
    'provider-name': 'aws',
    'source': 'https://github.com/terraform-aws-modules/terraform-aws-ec2-instance',
    'verified': True},
   'links': {'self': '/v2/modules/61'}},
  {'type': 'modules',
   'id': '1621',
   'attributes': {'download

# Providers

In [6]:
print(get_pagination(global_url_provider))

{'page-size': 15, 'current-page': 1, 'next-page': 2, 'prev-page': None, 'total-pages': 154, 'total-count': 2299}


# Modules

In [7]:
# Getting the list of all modules of v2 API
#list_modules = list(get_modules(global_url_modules, get_pagination(global_url_modules)['total-pages']))

# Next, giving pandas the list_modules dataframe
#df_modules = pd.DataFrame(list_modules)

# query the dataframe to get the list of modules with the provider name like aws, google and azurerm
#df_modules_aws = df_modules[df_modules['attributes']['provider-name'] == 'aws']
#df_modules_google = df_modules[df_modules['attributes']['provider-name'] == 'google']
#df_modules_azurerm = df_modules[df_modules['attributes']['provider-name'] == 'azurerm']